In [8]:
from dotenv import load_dotenv
from tavily import TavilyClient
import os
import sys

load_dotenv()
sys.path.insert(0, os.path.abspath('.'))

open_ai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")


In [9]:
from lib.state_machine import StateMachine, AgentState, ConversationTurn, SessionState
from lib.tools import GameRetrievalTool, RetrievalEvaluationTool, WebSearchTool
from lib.agents import VideoGameAgent

In [ ]:
agent = VideoGameAgent(db_path="./games_db")

In [ ]:
query_1 = "When was It Takes Two released?"
result_1 = agent.process_query(query_1)
agent.display_result(result_1)

In [ ]:
query_2 = "How do you play Raft to survive the first day?"
result_2 = agent.process_query(query_2)
agent.display_result(result_2)

In [ ]:
query_3 = "What platforms can you play Ravenswatch on?"
result_3 = agent.process_query(query_3)
agent.display_result(result_3)

In [ ]:
summary = agent.get_session_summary()

print("="*80)
print("SESSION SUMMARY")
print("="*80)
print(f"Session ID: {summary['session_id']}")
print(f"Created At: {summary['created_at']}")
print(f"Turns Completed: {summary['turns_completed']}")

print("\nConversation History:")
print("-"*80)

for entry in summary['conversation_history']:
    web_indicator = "🌐 WEB" if entry['web_search_used'] else "🗄️ DB"
    
    print(f"\nTurn {entry['turn_id']}: {entry['query']}")
    print(f"  Source: {entry['source']}")
    print(f"  Type: {web_indicator}")

print("="*80)

In [ ]:
state_machine = agent.state_machine

print("="*80)
print("STATE MACHINE CONFIGURATION")
print("="*80)

print("\nValid State Transitions:")
print("-"*80)

for from_state, to_states in state_machine.VALID_TRANSITIONS.items():
    to_state_names = [s.value for s in to_states]
    print(f"{from_state.value:20} → {', '.join(to_state_names)}")

print(f"\nCurrent State: {state_machine.session.current_state.value}")
print(f"Session ID: {state_machine.session.session_id}")
print("="*80)

In [ ]:
history = agent.state_machine.get_history()

retrieval_count = sum(1 for turn in history if not turn.web_search_used)
web_search_count = sum(1 for turn in history if turn.web_search_used)

print("="*80)
print("TOOL USAGE ANALYSIS")
print("="*80)

print(f"\nTotal Queries Processed: {len(history)}")
print(f"Local DB Retrievals: {retrieval_count}")
print(f"Web Searches: {web_search_count}")

print("\nTool Invocation Details:")
print("-"*80)

for turn in history:
    tools_used = ["retrieve_game", "evaluate_retrieval"]
    if turn.web_search_used:
        tools_used.append("game_web_search")
    
    print(f"\nTurn {turn.turn_id}: '{turn.query}'")
    print(f"  Tools Used: {', '.join(tools_used)}")
    print(f"  Confidence: {turn.confidence}")
    print(f"  Web Search: {'Yes' if turn.web_search_used else 'No'}")
    
    if turn.retrieval_results:
        print(f"  Retrieved {turn.retrieval_results['num_results']} candidates from DB")

print("="*80)

In [ ]:
print("="*80)
print("AGENT WORKFLOW TRACE")
print("="*80)

for turn in agent.state_machine.get_history():
    print(f"\n📌 Turn {turn.turn_id}: {turn.query}")
    print("-"*80)
    
    if turn.reasoning:
        reasoning_lines = turn.reasoning.split("\n")
        for line in reasoning_lines:
            if line.strip():
                print(f"  {line}")
    
    print(f"\n  ✓ Final Answer Source: {turn.source}")

print("="*80)